https://www.nitrc.org/plugins/mwiki/index.php/neurobureau:AthenaPipeline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("..")

from Code.data_generator import FMRIDataGenerator

import numpy as np
import pandas as pd
import os
import sys
from datetime import datetime

import tensorflow as tf

from tensorflow.keras.layers import Conv3D, MaxPool3D, TimeDistributed, Flatten, LSTM, Dense
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import CSVLogger

import tensorflow.keras as keras

import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [1]:
# ============================ DATA WORK ============================

file_num = '1'

# Dataframes
dataset_dir = "/pylon5/cc5614p/deopha32/fmri_images/model_data/"
model_train_data = pd.read_csv("/Users/vedantshirapure/Downloads/Diagnosing-ADHD-With-ConvLSTM/Data/training_data_{}".format(file_num) )

model_val_data = pd.read_csv("/Users/vedantshirapure/Downloads/Diagnosing-ADHD-With-ConvLSTM/Data/validatation_data_{}".format(file_num) )

# Dictionary of data values
partition = {'train': model_train_data['Image'].values, 
             'validation': model_val_data['Image'].values}

# Training Data
train_labels = {}
for index, row in model_train_data.iterrows():
    train_labels[row['Image']] = row['DX']
    
# Validation Data
val_labels = {}
for index, row in model_val_data.iterrows():
    val_labels[row['Image']] = row['DX']


from datetime import datetime
from tensorflow.keras.callbacks import CSVLogger  

NameError: name 'pd' is not defined

In [ ]:
# ============================ MODEL META ============================

epochs = 500
batch_size = 6
input_shape=(177,28,28,28,1)

train_steps_per_epoch = model_train_data.shape[0] // batch_size
validate_steps_per_epoch = model_val_data.shape[0] // batch_size

# Generators
training_generator = FMRIDataGenerator(partition['train'], train_labels, dataset_dir, batch_size)
validation_generator = FMRIDataGenerator(partition['validation'], val_labels, dataset_dir, batch_size)

curr_time = f'{datetime.now():%H-%M-%S%z_%m%d%Y}'
logger_path = "/pylon5/cc5614p/deopha32/Saved_Models/adhd-fmri-history_cv{num}_{time}.csv".format(num=file_num,time=curr_time)

csv_logger = CSVLogger(logger_path, append=True)

callbacks = [csv_logger]

import tensorflow as tf
from tensorflow.keras.models import Sequential  # Import Sequential model
from tensorflow.keras.layers import TimeDistributed, Conv3D, MaxPool3D, Flatten, LSTM, Dense  # Import necessary layers
from tensorflow.keras import optimizers  # Import optimizer

In [ ]:
# ============================ MODEL ARCHITECTURE ============================

with tf.device('/gpu:0'):
    cnn_lstm_model = Sequential()

    # Conv3D layer with TimeDistributed wrapper
    cnn_lstm_model.add(TimeDistributed(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu'),
                                       input_shape=input_shape, name="Input_Conv_Layer"))

    # MaxPool3D layer with TimeDistributed wrapper
    cnn_lstm_model.add(TimeDistributed(MaxPool3D(pool_size=(2, 2, 2),
                                                  strides=(2, 2, 2),
                                                  padding='valid'),
                                       name="Pool_Layer_1"))

    # Flatten layer with TimeDistributed wrapper
    cnn_lstm_model.add(TimeDistributed(Flatten(), name="Flatten_Layer"))
    
# LSTM layer on CPU
with tf.device('/cpu:0'):
    cnn_lstm_model.add(LSTM(10, dropout=0.3, recurrent_dropout=0.3, name="LSTM_Layer"))

# Output layer with Dense layer on GPU
with tf.device('/gpu:0'):
    cnn_lstm_model.add(Dense(1, activation='sigmoid', name="Output_Dense_Layer"))

# Compile the model
cnn_lstm_model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                       loss='binary_crossentropy',
                       metrics=['accuracy'])


# https://stackoverflow.com/a/46216013/9221241
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    number_size = 4.0
    if K.floatx() == 'float16':
         number_size = 2.0
    if K.floatx() == 'float64':
         number_size = 8.0

    total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

# Misc. 

In [ ]:
# https://stackoverflow.com/a/46216013/9221241
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    number_size = 4.0
    if K.floatx() == 'float16':
         number_size = 2.0
    if K.floatx() == 'float64':
         number_size = 8.0

    total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

In [ ]:
#get_model_memory_usage(32, cnn_lstm_model)

AttributeError: 'TimeDistributed' object has no attribute 'output_shape'

In [ ]:
cnn_lstm_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Conv_Layer                │ (None, 177, 26, 26,    │         1,792 │
│ (TimeDistributed)               │ 26, 64)                │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool_Layer_1 (TimeDistributed)  │ (None, 177, 13, 13,    │             0 │
│                                 │ 13, 64)                │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten_Layer (TimeDistributed) │ (None, 177, 140608)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM_Layer (LSTM)               │ (None, 10)             │     5,624,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Dense_Layer (Dense)      │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,626,563 (21.46 MB)

 Trainable params: 5,626,563 (21.46 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# import numpy as np
# from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
#                              f1_score, roc_auc_score, confusion_matrix, 
#                              ConfusionMatrixDisplay, roc_curve, auc)
# import matplotlib.pyplot as plt

# # ============================ MODEL TRAINING ============================
# # Make sure to capture the training history
# history = cnn_lstm_model.fit(
#     training_generator,
#     steps_per_epoch=train_steps_per_epoch,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=validate_steps_per_epoch,
#     callbacks=callbacks,
#     verbose=1
# )

# # ============================ EVALUATION METRICS ============================
# # Generate predictions
# print("Generating predictions...")
# y_true = np.array([val_labels[img_id] for img_id in partition['validation']])
# y_pred_probs = cnn_lstm_model.predict(validation_generator, verbose=1)
# y_pred = (y_pred_probs >= 0.5).astype(int)

# # Calculate metrics
# accuracy = accuracy_score(y_true, y_pred)
# precision = precision_score(y_true, y_pred)
# recall = recall_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)
# roc_auc = roc_auc_score(y_true, y_pred_probs)

# print(f"\nValidation Metrics:")
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")
# print(f"ROC AUC: {roc_auc:.4f}")

# # ============================ VISUALIZATIONS ============================
# # Plot training history
# plt.figure(figsize=(15, 6))

# # Loss plot
# plt.subplot(1, 2, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend()

# # Accuracy plot
# plt.subplot(1, 2, 2)
# plt.plot(history.history['accuracy'], label='Train Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend()

# plt.tight_layout()
# plt.savefig('/pylon5/cc5614p/deopha32/Saved_Models/training_history.png')
# plt.close()

# # Confusion Matrix
# cm = confusion_matrix(y_true, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
#                               display_labels=['Control', 'ADHD'])
# disp.plot(cmap=plt.cm.Blues)
# plt.title('Confusion Matrix')
# plt.savefig('/pylon5/cc5614p/deopha32/Saved_Models/confusion_matrix.png')
# plt.close()

# # ROC Curve
# fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
# roc_auc = auc(fpr, tpr)

# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, 
#          label=f'ROC curve (area = {roc_auc:.2f})')
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.savefig('/pylon5/cc5614p/deopha32/Saved_Models/roc_curve.png')
# plt.close()

# # Save metrics to file
# with open('/pylon5/cc5614p/deopha32/Saved_Models/evaluation_metrics.txt', 'w') as f:
#     f.write(f'Accuracy: {accuracy:.4f}\n')
#     f.write(f'Precision: {precision:.4f}\n')
#     f.write(f'Recall: {recall:.4f}\n')
#     f.write(f'F1 Score: {f1:.4f}\n')
#     f.write(f'ROC AUC: {roc_auc:.4f}\n')

# print("All evaluation metrics and plots saved successfully!")

# ============================ EVALUATION METRICS ============================
# Generate predictions
print("Generating predictions...")
y_true = np.array([val_labels[img_id] for img_id in partition['validation']])
y_pred_probs = cnn_lstm_model.predict(validation_generator, verbose=1)
y_pred = (y_pred_probs >= 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Print accuracy
print(f"Accuracy: {accuracy:.4f}")

Generating predictions...


FileNotFoundError: No such file or no access: '/pylon5/cc5614p/deopha32/fmri_images/model_data/Peking_1_sfnwmrda2443191_session_1_rest_1.nii.gz'